# Import libraries

In [ ]:
!pip install einops timm sentencepiece

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import torch
from transformers import AutoModel, AutoTokenizer
torch.set_grad_enabled(False)

# Instantiate model

In [ ]:
# Instantiate InternLM-XComposer model. via
# https://github.com/InternLM/InternLM-XComposer
model = AutoModel.from_pretrained('internlm/internlm-xcomposer-7b', trust_remote_code=True).cuda().eval()
tokenizer = AutoTokenizer.from_pretrained('internlm/internlm-xcomposer-7b', trust_remote_code=True)
model.tokenizer = tokenizer

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## DISARM

In [ ]:
DISARM_test_all_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_Qwen-VL-Chat.csv')
DISARM_test_all_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_InternLM-XComposer.csv')
DISARM_test_all_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_llava-v1.5-13b.csv')
DISARM_test_all_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/DISARM_test_all_captioned_BLIP-2.csv')

# Set prompts and define a function to call the model

In [ ]:
def prompt_vanilla(entity):
    return inspect.cleandoc(f"""
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
def prompt_with_OCR(entity, OCR):
    return inspect.cleandoc(f"""
    Text on this meme: \"\"\"
    {OCR}
    \"\"\"
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
def prompt_with_caption(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    Does this meme harmfully target {entity}?
    yes: a social entity is subjected to some form of ill-treatment such as mental abuse, psycho-physiological injury, proprietary damage, emotional disturbance, or public image damage, based on their background (bias, social background, educational background, etc.) by a meme author.
    no: any benign mention (or depiction) of a social entity via humour, limerick, harmless pun or any content that does not cause distress.
    Constraint: Without using any other words, answer either yes or no.""")

In [ ]:
# Use InternLM-XComposer for the inference. via
# https://github.com/InternLM/InternLM-XComposer
def get_prediction(image, prompt):
    response = model.generate(prompt, image)
    return response

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x == 'yes':
        return 'harmful'
    elif x == 'no':
        return 'not harmful'
    elif 'yes' in x and 'no' not in x:
        return 'harmful'
    elif 'no' in x and 'yes' not in x and 'not' not in x:
        return 'not harmful'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [VANILLA]

In [ ]:
DISARM_test_all_grounding_absent_prompt_vanilla = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_vanilla_images = DISARM_test_all_grounding_absent_prompt_vanilla['image'].values
DISARM_test_all_grounding_absent_prompt_vanilla_entities = DISARM_test_all_grounding_absent_prompt_vanilla['target'].values
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = [get_prediction(image, prompt_vanilla(entity)) for image, entity in zip(DISARM_test_all_grounding_absent_prompt_vanilla_images, DISARM_test_all_grounding_absent_prompt_vanilla_entities)]
DISARM_test_all_grounding_absent_prompt_vanilla.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_prompt[VANILLA]_prediction[InternLM-XComposer].csv', index=False)
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_vanilla['prediction'] = DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_vanilla['labels'].values, DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_vanilla['labels'].values, DISARM_test_all_grounding_absent_prompt_vanilla['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    612
Name: prediction, dtype: int64
0
0.32599118942731276
              precision    recall  f1-score   support

     harmful       0.00      0.00      0.00       316
 not harmful       0.48      1.00      0.65       296

    accuracy                           0.48       612
   macro avg       0.24      0.50      0.33       612
weighted avg       0.23      0.48      0.32       612



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GROUNDING [ABSENT] & PROMPT [OCR]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_OCR = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_OCR_images = DISARM_test_all_grounding_absent_prompt_with_OCR['image'].values
DISARM_test_all_grounding_absent_prompt_with_OCR_entities = DISARM_test_all_grounding_absent_prompt_with_OCR['target'].values
DISARM_test_all_grounding_absent_prompt_with_OCR_texts = DISARM_test_all_grounding_absent_prompt_with_OCR['extracted_text'].values
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = [get_prediction(image, prompt_with_OCR(entity, text)) for image, entity, text in zip(DISARM_test_all_grounding_absent_prompt_with_OCR_images, DISARM_test_all_grounding_absent_prompt_with_OCR_entities, DISARM_test_all_grounding_absent_prompt_with_OCR_texts)]
DISARM_test_all_grounding_absent_prompt_with_OCR.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_prompt[OCR]_prediction[InternLM-XComposer].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'] = DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_OCR['labels'].values, DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_OCR['labels'].values, DISARM_test_all_grounding_absent_prompt_with_OCR['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    612
Name: prediction, dtype: int64
0
0.32599118942731276
              precision    recall  f1-score   support

     harmful       0.00      0.00      0.00       316
 not harmful       0.48      1.00      0.65       296

    accuracy                           0.48       612
   macro avg       0.24      0.50      0.33       612
weighted avg       0.23      0.48      0.32       612



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat = DISARM_test_all_captioned_Qwen_VL_Chat.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    593
harmful         19
Name: prediction, dtype: int64
0
0.3444655238990648
              precision    recall  f1-score   support

     harmful       0.42      0.03      0.05       316
 not harmful       0.48      0.96      0.64       296

    accuracy                           0.48       612
   macro avg       0.45      0.49      0.34       612
weighted avg       0.45      0.48      0.33       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer = DISARM_test_all_captioned_InternLM_XComposer.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_images, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    590
harmful         22
Name: prediction, dtype: int64
0
0.3828255439646306
              precision    recall  f1-score   support

     harmful       0.82      0.06      0.11       316
 not harmful       0.49      0.99      0.66       296

    accuracy                           0.51       612
   macro avg       0.66      0.52      0.38       612
weighted avg       0.66      0.51      0.37       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_llava = DISARM_test_all_captioned_llava.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_llava_images = DISARM_test_all_grounding_absent_prompt_with_caption_llava['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities = DISARM_test_all_grounding_absent_prompt_with_caption_llava['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions = DISARM_test_all_grounding_absent_prompt_with_caption_llava['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_llava_images, DISARM_test_all_grounding_absent_prompt_with_caption_llava_entities, DISARM_test_all_grounding_absent_prompt_with_caption_llava_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_llava['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    607
harmful          5
Name: prediction, dtype: int64
0
0.33914987425093923
              precision    recall  f1-score   support

     harmful       0.80      0.01      0.02       316
 not harmful       0.49      1.00      0.65       296

    accuracy                           0.49       612
   macro avg       0.64      0.50      0.34       612
weighted avg       0.65      0.49      0.33       612



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2 = DISARM_test_all_captioned_BLIP_2.copy(deep=True)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['image'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['target'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_images, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_entities, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2_captions)]
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/DISARM_test_all_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[InternLM-XComposer].csv', index=False)
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['harmful', 'not harmful']))
print(f1_score(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful'], average='macro'))
print(classification_report(DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['labels'].values, DISARM_test_all_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['harmful', 'not harmful']))

not harmful    612
Name: prediction, dtype: int64
0
0.32599118942731276
              precision    recall  f1-score   support

     harmful       0.00      0.00      0.00       316
 not harmful       0.48      1.00      0.65       296

    accuracy                           0.48       612
   macro avg       0.24      0.50      0.33       612
weighted avg       0.23      0.48      0.32       612



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
